# AgnoAgent Notebook — Overview & Run Guide

## 1. Structure (top to bottom)

| Block | Purpose |
|-------|---------|
| **0. Install** | Install agno, openai, google-genai, mcp, fastapi, sqlalchemy |
| **1. Basic Agent** | OpenRouter + InMemoryDb chat agent; run "Hello" |
| **2. Agent + Tools + Structured Output** | `add` tool; Response schema (chat / answer / tools_used) |
| **3. Interview Agent (simple)** | Single agent: technical + Data Scientist context; one question per turn; InterviewTurn (question, type, expected key points) |
| **4. Full Interview System** | RAG knowledge base, State, tools, Coach Agent, Team, Session Summary, Workflow |

## 2. What each block does

### Block 1: Install
- Install `agno`, `openai`, `google-genai`, `mcp`, `fastapi`, `sqlalchemy`. Run once.

### Block 2: Basic Agent (OpenRouter + memory)
- **OpenRouter** `gpt-4o-mini`, **InMemoryDb** for history. `BasicAgent` replies as "You are a helpful AI Assistant". Run `agent.run("Hello, how are you?")` for a reply.

### Block 3: Agent + Tools + Structured Output
- **Tool**: `add(a, b)`. **Structured output**: Pydantic `Response` with `chat`, `answer`, `tools_used`. Ask "What's 10 + 2?" to trigger `add` and structured reply.

### Block 4: Interview Agent (Phase 1, single agent)
- **Context**: technical interview, Data Scientist role (in instructions). **Output**: `InterviewTurn` (question, type, expected key points). No RAG/State/Team. Run `interview_agent.run("I'm ready. Please give me the first question.")`.

### Block 5: Full system — RAG knowledge base
- Chunk **ds notes.markdown** by `##` into sections. Build **ChromaDb** vector store, **Knowledge** for topic-based questions. Set `DS_NOTES_PATH` to your local path.

### Block 6: State + Tools
- **State**: `used_section_titles`, `round`, `session_start_time`, `interview_ended`. **Tools**: `get_current_time`, `pick_topic_from_rag` (pick unused topic for next question), `end_interview`. Used by Interviewer Agent.

### Block 7: Coach Agent + request_feedback
- **Coach**: feedback/scoring from question + candidate answer + RAG reference (score, covered/missing points, suggestion). **request_feedback** tool: Interviewer calls it after an answer; it queries RAG, calls Coach, returns formatted feedback.

### Block 8: Interviewer Agent + Team
- **Interviewer**: asks questions, runs flow; uses all tools + State + RAG; outputs `InterviewTurn` (or `session_summary` when ending). **Team**: Interviewer + Coach; Coach is invoked via `request_feedback`, not Team routing.

### Block 9: Session Summary + Workflow
- **Session Summary**: when user says "end", run `generate_session_summary(agent)` to produce a short recap from conversation history. **Workflow**: set context → multi-round Q&A → user says end → `end_interview` → Session Summary.

### Block 10: How to run
- **Single-run demo**: run "ask first question" → "candidate answer + request feedback" → "end interview + Session Summary".
- **Interactive**: uncomment `run_interview_workflow(interviewer_agent, max_rounds=5)`, type answers in terminal; say "end" to finish and print summary.

## 3. Run order

1. **Environment**: Python, recommended env (e.g. `agno_env`). **API Key**: `OPENROUTER_API_KEY` in env or fallback in code.
2. Run **Install**, then **Basic Agent** and **Agent + Tools**.
3. For **full interview**: have `ds notes.markdown`, set `DS_NOTES_PATH`, then run RAG, State+Tools, Coach, Interviewer, Team, Summary, Workflow in order.
4. For **simple interview** only: run up to Block 4; no RAG/ChromaDb.
5. **Interactive**: after full setup, run `run_interview_workflow(...)`, say "end" to end.

## 4. Dependencies & paths

- **ChromaDb**: required for full system; run `!pip install chromadb` if needed.
- **OPENROUTER_API_KEY**: must be valid.
- **DS_NOTES_PATH**: path to your `ds notes.markdown` for RAG.

In [ ]:
!pip install agno openai google-genai mcp fastapi sqlalchemy

# Build Agno Agent with OpenRouter

Build a basic agent, then expand the agent with other Agno modules

Ask me for an OpenRouter API key

- [Agno Docs: Building Agents](https://docs.agno.com/basics/agents/building-agents)
- [Agno Docs: OpenRouter](https://docs.agno.com/integrations/models/gateways/openrouter/overview)
- [Cookbook/Examples](https://github.com/agno-agi/agno/tree/main/cookbook)

Set `OPENROUTER_API_KEY` in your environment, or in the code cells below.

### Models
[See all OpenRouter models](https://openrouter.ai/models)
- Search by price, category, parameters allowed, etc
- **Price**: Use Free models as able, but you might need some of the paid ones for more complex tasks.
    - Try to use ones that cost less than $1.00 per 1M / output tokens

In [ ]:
import os
from textwrap import dedent
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb

# For local run: use env var OPENROUTER_API_KEY if set, otherwise use key below
if 'OPENROUTER_API_KEY' not in os.environ:
    os.environ['OPENROUTER_API_KEY'] = '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'
DB = InMemoryDb()
SESSION_ID = 'session001'

agent = Agent(
    name="BasicAgent",
    model=OpenRouter(
        id='openai/gpt-4o-mini',
        api_key=os.environ['OPENROUTER_API_KEY'],
    ),
    instructions=dedent(
        """
        You are a helpful AI Assistant
        """
    ),
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
)

In [ ]:
response = agent.run("Hello, how are you?")
print(response.content)

## Tasks
Review all the modules of Agno SDK and build an over-engineered Agent that demonstrates key concepts and features

1. [Agno Context Engineering](https://docs.agno.com/context/engineering/overview)

2. [Structured Outputs with Pydantic](https://docs.agno.com/basics/input-output/overview#structured-output)

2. [State Management / Agentic State](https://docs.agno.com/context/state/overview)

3. [Knowledge / Agentic RAG](https://docs.agno.com/context/knowledge/overview)

4. [Session Summary Agent](https://docs.agno.com/context/knowledge/overview)

5. [Tools and Toolkits](https://docs.agno.com/integrations/toolkits/overview)

6. [Reasoning](https://docs.agno.com/features/reasoning/overview)

7. [Build a second agent and use as a `Team`](https://docs.agno.com/basics/teams/overview)

8. [Build `Steps` and use in a `Workflow`](https://docs.agno.com/basics/workflows/building-workflows)

> Does not need to especially sophisticated, and does not need to be an especially useful agent, just work to get the different elements integrated

For example, add a basic tool and structured outputs to your agent:

### Agent with Tool and Structured Output

In [ ]:
import os
from textwrap import dedent
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb
from agno.tools import tool
from pydantic import BaseModel, Field
from typing_extensions import Optional, List

if 'OPENROUTER_API_KEY' not in os.environ:
    os.environ['OPENROUTER_API_KEY'] = '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'
DB = InMemoryDb()
SESSION_ID = 'session001'

# Create simple arithmetic tool
@tool
def add(a: int, b: int) -> int:
    """ A tool that adds integers together """
    return a + b

# Create structured output schema with Typed data
class Response(BaseModel):
    """
    A response to a user input that includes:
    - a normal LLM chat response
    - a numerical answer if the user input required math tools
    - a list of tools used (names of tools)
    """
    chat: str = Field(..., description="The chat response to the user input")
    answer: Optional[int] = Field(
        None,
        description="Numerical answer to the user input if it required math tools"
    )
    tools_used: List[str]

# Create agent
agent = Agent(
    name="BasicAgent",
    model=OpenRouter(
        id='openai/gpt-4o-mini',
        api_key=os.environ['OPENROUTER_API_KEY'],
    ),
    instructions=dedent(
        """
        You are a helpful AI Assistant. You can add two numbers together
        using a tool, which is more accurate than LLM native arithmetic.
        """
    ),
    tools=[add],
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
    output_schema=Response,
    use_json_mode=True,  # Helps with some endpoint/model compatibility issues
    retries=3
)

# We'll skip the pretty print response and get the data as a dict/json
run = agent.run("Hey! What's 10 + 2?")  # to display output
print(run.content.model_dump())

# Now we can access response items programatically, enabling all sorts of follow on operations
print("\nAccess individual fields from response:")
print(f"{type(run.content.chat)}: {run.content.chat}")
print(f"{type(run.content.answer)}: {run.content.answer}")
print(f"{type(run.content.tools_used)}: {run.content.tools_used}")

### Interview Agent (Phase 1: Single Agent + Structured Output + Context)

Mock interview coach: one agent asks questions based on **context** (interview type, role) and returns **structured output** (current question + type + expected key points). No RAG/State/Team yet—just get the conversation working.

In [ ]:
import os
from textwrap import dedent
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from agno.db.in_memory import InMemoryDb
from pydantic import BaseModel, Field
from typing import List

# API key: set OPENROUTER_API_KEY in env, or use fallback below for local run
if "OPENROUTER_API_KEY" not in os.environ:
    os.environ["OPENROUTER_API_KEY"] = "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"

DB = InMemoryDb()
SESSION_ID = "interview_session_001"

# --- Context (simple context engineering) ---
INTERVIEW_TYPE = "technical"
ROLE = "Data Scientist"

# --- Structured output: one interview turn = question + type + expected key points ---
class InterviewTurn(BaseModel):
    """One interviewer turn: the question to ask and what to look for in the answer."""
    current_question: str = Field(..., description="The interview question to ask the candidate")
    question_type: str = Field(..., description="e.g. technical, behavioral, system_design")
    expected_key_points: List[str] = Field(..., description="Key points or themes to look for in a good answer")

# --- Interview Agent: single agent, structured output, context in instructions ---
interview_agent = Agent(
    name="InterviewAgent",
    model=OpenRouter(
        id="openai/gpt-4o-mini",
        api_key=os.environ["OPENROUTER_API_KEY"],
    ),
    instructions=dedent(f"""
        You are a professional interviewer conducting a mock data science interview.
        Interview type: {INTERVIEW_TYPE}.
        Role: {ROLE}.
        Focus on data science topics: statistics, ML, Python/pandas, SQL, A/B testing, metrics, etc.
        Ask exactly ONE question per turn. Be concise and professional.
        Output your question and the expected key points in the required structured format.
        """),
    db=DB,
    session_id=SESSION_ID,
    add_history_to_context=True,
    output_schema=InterviewTurn,
    use_json_mode=True,
    retries=3,
)

# --- Run: ask for the first question ---
run = interview_agent.run("I'm ready. Please give me the first question.")
print("Raw output (dict):", run.content.model_dump())
print("\n--- Formatted ---")
print(f"Question: {run.content.current_question}")
print(f"Type: {run.content.question_type}")
print(f"Expected key points: {run.content.expected_key_points}")

---
## Interview Agent (Full): State + RAG + Tools + Team + Session Summary + Workflow

- **State**: Track `used_section_titles`, `round`, `session_start_time`, `interview_ended`
- **RAG**: DS notes markdown chunked by `##`, used for question generation and feedback reference
- **Tools**: `get_current_time`, `pick_topic_from_rag`, `end_interview`, `request_feedback` (calls Coach)
- **Team**: Interviewer (questions / flow) + Coach (feedback / scoring)
- **Session Summary**: Triggered on "end interview", included in structured output or displayed separately
- **Workflow**: Set context → multi-round Q&A (team) → summary

In [ ]:
# Install chromadb for vector store (run if ModuleNotFoundError: chromadb)
!pip install chromadb

In [ ]:
# --- 1. RAG: Build knowledge base from DS notes (chunk by ## sections) ---
import os
import re
from pathlib import Path

if "OPENROUTER_API_KEY" not in os.environ:
    os.environ["OPENROUTER_API_KEY"] = "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"  # set your key
# ChromaDb default embedder may need OPENAI_API_KEY; or pass a custom embedder to ChromaDb
DS_NOTES_PATH = "/Users/joy/Desktop/ds notes.markdown"

def parse_markdown_by_h2(path: str) -> list[tuple[str, str]]:
    """Parse markdown file into (section_title, content) pairs by ## headers."""
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    # Split by ## but keep the header line with the content
    pattern = re.compile(r"^##\s+(.+)$", re.MULTILINE)
    parts = pattern.split(text)
    # parts[0] is content before first ##; then [title1, content1, title2, content2, ...]
    sections = []
    if len(parts) <= 1:
        if text.strip():
            sections.append(("general", text))
        return sections
    # Skip leading content without ## (e.g. first line or intro)
    i = 1
    while i + 1 < len(parts):
        title, content = parts[i].strip(), (parts[i + 1] or "").strip()
        if title or content:
            sections.append((title or "section", content or "(no content)"))
        i += 2
    return sections

sections = parse_markdown_by_h2(DS_NOTES_PATH)
print(f"Parsed {len(sections)} sections (##). First 3 titles: {[t for t, _ in sections[:3]]}")

In [ ]:
from agno.knowledge import Knowledge
from agno.knowledge.embedder.openai import OpenAIEmbedder
from agno.vectordb.chroma import ChromaDb
from chromadb import PersistentClient
from pathlib import Path
import os

CHROMA_PATH = "./tmp_interview_chroma"
COLLECTION = "ds_notes_emb1536"

Path(CHROMA_PATH).mkdir(parents=True, exist_ok=True)

EMBED_MODEL = "text-embedding-3-small"  # 1536 dim
embedder = OpenAIEmbedder(
    id=EMBED_MODEL,
    api_key=os.environ["OPENROUTER_API_KEY"],
    base_url="https://openrouter.ai/api/v1",
)

# 1. Remove existing collection if present
client = PersistentClient(path=CHROMA_PATH)
if COLLECTION in [c.name for c in client.list_collections()]:
    client.delete_collection(COLLECTION)

# 2. Create new collection (embedding dim fixed here)
coll = client.get_or_create_collection(
    COLLECTION,
    metadata={"hnsw:space": "cosine"},
)

# 3. Upsert sections
inserted = skipped = failed = 0
for i, (title, content) in enumerate(sections):
    if not isinstance(content, str):
        skipped += 1
        continue
    if len(content.strip()) < 30 or content.strip().startswith("!["):
        skipped += 1
        continue

    try:
        emb = embedder.get_embedding(content)
        if not emb:
            failed += 1
            continue

        coll.upsert(
            ids=[f"ds_sec_{i}"],
            embeddings=[emb],
            documents=[content],
            metadatas=[{"section_title": title}],
        )
        inserted += 1
    except Exception as e:
        failed += 1
        print("Insert fail:", e)

print(f"Inserted={inserted}, skipped={skipped}, failed={failed}")

# 4. Wire Knowledge / ChromaDb to this collection
vector_db = ChromaDb(
    collection=COLLECTION,
    path=CHROMA_PATH,
    embedder=embedder,
    persistent_client=True,
)

knowledge = Knowledge(
    name=COLLECTION,
    vector_db=vector_db,
    max_results=5,
)


In [ ]:
q_emb = embedder.get_embedding("what is overfitting")

res = coll.query(
    query_embeddings=[q_emb],  # use this embedding for Chroma query
    n_results=3,
)

print(res["documents"])


### 2. State + Tools

- **State**: `used_section_titles`, `round`, `session_start_time`, `interview_ended`
- **Tools**: `get_current_time`, `pick_topic_from_rag`, `end_interview`; Coach is invoked via Team or a tool.

In [ ]:
# --- State schema and tools (use run_context for state) ---
from datetime import datetime
from agno.run import RunContext
from agno.tools import tool

# Default session state for interview
INTERVIEW_SESSION_STATE = {
    "used_section_titles": [],
    "round": 0,
    "session_start_time": None,
    "interview_ended": False,
}

@tool
def get_current_time(run_context: RunContext) -> str:
    """Return current time. Use when starting a round or reporting duration."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def pick_topic_from_rag(run_context: RunContext, query: str = "data science interview topic") -> str:
    """Retrieve one topic from the knowledge base that has NOT been used yet. Use this to pick the next question topic. Returns the section content; you should generate a question based on it."""
    used = run_context.session_state.get("used_section_titles") or []
    docs = knowledge.search(query=query, max_results=10)
    for doc in docs:
        title = doc.meta_data.get("section_title") or doc.name
        if title and title not in used:
            run_context.session_state.setdefault("used_section_titles", []).append(title)
            return f"[Section: {title}]\n{doc.content}"
    return "No unused topics left; you may wrap up the interview or repeat from a broader search."

@tool
def end_interview(run_context: RunContext) -> str:
    """Call when the candidate says they want to end or when you decide to finish. Marks interview as ended so summary can be generated."""
    run_context.session_state["interview_ended"] = True
    return "Interview ended. Session summary will be generated."

### 3. Coach Agent (feedback/scoring) + request_feedback tool

Coach evaluates candidate answers using RAG-retrieved reference; Interviewer calls Coach via tool when feedback is requested.

In [ ]:
# --- Coach Agent: feedback/scoring; structured output ---
from textwrap import dedent
from agno.agent import Agent
from agno.models.openrouter import OpenRouter
from pydantic import BaseModel, Field
from typing import List

class FeedbackOutput(BaseModel):
    """Structured feedback for one answer."""
    score_1_to_5: int = Field(..., ge=1, le=5, description="Score from 1 to 5")
    key_points_covered: List[str] = Field(..., description="Key points the candidate got right")
    missing_points: List[str] = Field(..., description="Important points missed (from reference)")
    suggestion: str = Field(..., description="One concise suggestion to improve")

coach_agent = Agent(
    name="CoachAgent",
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
    instructions=dedent("""
        You are an interview coach. Given the interview question, the candidate's answer, and the reference knowledge,
        provide structured feedback: score (1-5), key points covered, missing points, and one suggestion.
        Be fair and concise. Output in the required structured format.
    """),
    output_schema=FeedbackOutput,
    use_json_mode=True,
    retries=2,
)

In [ ]:
# --- request_feedback tool: calls Coach with RAG reference ---
def make_request_feedback_tool(coach_agent, knowledge):
    @tool
    def request_feedback(run_context: RunContext, question: str, candidate_answer: str) -> str:
        """Ask the coach for feedback on the candidate's answer. Provide the question and the candidate's answer. Use after the candidate has answered a question."""
        ref_docs = knowledge.search(query=question, max_results=3)
        reference = "\n\n".join(d.content for d in ref_docs) if ref_docs else "No reference."
        prompt = f"Question: {question}\n\nCandidate answer: {candidate_answer}\n\nReference knowledge:\n{reference}\n\nGive structured feedback (score, key points covered, missing points, suggestion)."
        out = coach_agent.run(prompt)
        if hasattr(out.content, "model_dump"):
            d = out.content.model_dump()
            return f"Score: {d.get('score_1_to_5')}/5 | Covered: {d.get('key_points_covered')} | Missing: {d.get('missing_points')} | Suggestion: {d.get('suggestion')}"
        return str(out.content)

    return request_feedback

request_feedback = make_request_feedback_tool(coach_agent, knowledge)

### 4. Interviewer Agent + Team

Interviewer: questions / flow, tools + state + RAG. Coach: feedback / scoring. Team = Interviewer + Coach (interviewer calls coach via `request_feedback` tool).

In [ ]:
# --- Interviewer Agent: questions, control, tools, state, RAG ---
from textwrap import dedent
from agno.agent import Agent
from agno.db.in_memory import InMemoryDb
from agno.models.openrouter import OpenRouter
from pydantic import BaseModel, Field
from typing import List, Optional

INTERVIEW_TYPE = "technical"
ROLE = "Data Scientist"
SESSION_ID = "interview_full_001"
DB = InMemoryDb()

class InterviewTurn(BaseModel):
    """One interviewer turn: question + type + expected key points, OR session_summary when ending."""
    current_question: Optional[str] = Field(None, description="The interview question to ask the candidate")
    question_type: Optional[str] = Field(None, description="e.g. technical, behavioral")
    expected_key_points: Optional[List[str]] = Field(None, description="Key points to look for in a good answer")
    session_summary: Optional[str] = Field(None, description="If interview ended, summary of the session")

interviewer_agent = Agent(
    name="InterviewAgent",
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
    instructions=dedent(f"""
        You are a professional interviewer for a mock data science interview.
        Interview type: {INTERVIEW_TYPE}. Role: {ROLE}.
        Use tools: get_current_time to track rounds; pick_topic_from_rag to get an unused topic and generate ONE question; request_feedback(question, candidate_answer) when the candidate has answered and you want to give feedback; end_interview when the candidate wants to stop or you finish.
        Ask exactly ONE question per turn. Be concise. Output in the required structured format.
        If the candidate says "end" or "end interview", call end_interview and then output session_summary as a brief recap of the session.
        """),
    tools=[get_current_time, pick_topic_from_rag, end_interview, request_feedback],
    db=DB,
    session_id=SESSION_ID,
    session_state=INTERVIEW_SESSION_STATE,
    add_session_state_to_context=True,
    add_history_to_context=True,
    knowledge=knowledge,
    search_knowledge=True,
    output_schema=InterviewTurn,
    use_json_mode=True,
    retries=3,
)

In [ ]:
# --- Team: Interviewer (lead) + Coach (called via request_feedback tool) ---
from agno.team import Team

interview_team = Team(
    name="InterviewTeam",
    members=[interviewer_agent, coach_agent],
    model=OpenRouter(id="openai/gpt-4o-mini", api_key=os.environ.get("OPENROUTER_API_KEY")),
)

# Note: Coach is invoked by the interviewer through the request_feedback tool, not by Team routing.
# Team allows both agents to share the same run; the interviewer has the tools and leads the flow.

### 5. Session Summary + Workflow

- **Session Summary**: Triggered when interview ends; generate and display (or include in structured output).
- **Workflow**: Set context → multi-round Q&A (run interviewer until end) → generate summary.

In [ ]:
# --- Session Summary: generate when interview_ended (run agent with summary prompt; history in context) ---
def generate_session_summary(agent) -> str:
    """Generate session summary. Call after end_interview. Agent has conversation in context."""
    summary_prompt = "The interview has ended. Based on our conversation, provide a brief session summary (3-5 sentences): topics covered, number of questions, and overall impression. Put the summary in session_summary if you use structured output, or reply with the summary text."
    out = agent.run(summary_prompt)
    if hasattr(out.content, "session_summary") and out.content.session_summary:
        return out.content.session_summary
    if hasattr(out.content, "current_question"):
        return out.content.current_question  # model might put summary in this field
    return str(out.content)

### 6. Workflow: run interview (set context → multi-round → summary)

Single run example: one question + one answer + optional feedback. For full loop, run the next cell repeatedly or use the helper loop below.

In [ ]:
# --- Workflow: one round (ask for first question) ---
run = interviewer_agent.run("I'm ready. Please give me the first question (use pick_topic_from_rag to choose an unused topic).")
print("--- Interview Turn ---")
if hasattr(run.content, "model_dump"):
    d = run.content.model_dump()
    for k, v in d.items():
        if v is not None:
            print(f"{k}: {v}")
else:
    print(run.content)

In [ ]:
# --- Example: candidate answers, then request feedback (interviewer can call request_feedback tool) ---
# Replace with the actual question from the previous run
last_question = run.content.current_question if hasattr(run.content, "current_question") else "Explain bias and variance."
candidate_answer = "Bias is the difference between model prediction and true value; high bias means underfitting. Variance is sensitivity to data; high variance means overfitting. They trade off."
run2 = interviewer_agent.run(f"I'll answer: {candidate_answer}. Please give me feedback on my answer.")
print("--- Response (may include feedback from Coach) ---")
print(run2.content if not hasattr(run2.content, "model_dump") else run2.content.model_dump())

In [ ]:
# --- End interview and get Session Summary ---
run_end = interviewer_agent.run("I want to end the interview. Please wrap up.")
print("--- End turn ---")
print(run_end.content.model_dump() if hasattr(run_end.content, "model_dump") else run_end.content)

# Generate and display session summary
summary = generate_session_summary(interviewer_agent)
print("\n--- Session Summary ---")
print(summary)

In [ ]:
# --- Workflow loop: multi-round until "end" → summary ---
def run_interview_workflow(agent, max_rounds: int = 5):
    """Run interview: multi-round until user says end or max_rounds. Then print summary."""
    print("Interview started. Say 'end' or 'end interview' to finish.\n")
    for r in range(max_rounds):
        user_input = input(f"Round {r+1} (you): ").strip()
        if not user_input:
            continue
        if user_input.lower() in ("end", "end interview"):
            _ = agent.run("I want to end the interview. Please wrap up.")
            break
        out = agent.run(user_input)
        print("Agent:", getattr(out.content, "current_question", out.content))
    print("\n--- Session Summary ---")
    print(generate_session_summary(agent))

# Uncomment to run interactive loop:
# run_interview_workflow(interviewer_agent, max_rounds=5)